In [1]:
import spacy
from spacy.matcher import Matcher
import re
import pandas as pd
import sys, fitz
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
import pymongo
from pymongo import MongoClient
import os
import docx2txt
import pickle
from nltk.tokenize import word_tokenize
import phonenumbers
from phonenumbers import geocoder
from phonenumbers import carrier
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import string
import re
import json
import pickle
import os
import sys,fitz

In [2]:

class resumeExtraction:
    def __init__(self):
        self.STOPWORDS = set(stopwords.words('english')+['``',"''"])
        # Education Degrees
        self.EDUCATION = [
                    'BE', 'BSC', 'BS', 
                    'ME','MS','BCOM','BCS','BCA','MCA',
                    'BTECH', 'MTECH','DIPLOMA','12TH','10TH',
                    'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII', 'XTH','XIITH','FE','SE','TE'
                ]
        self.data= pd.read_csv(r"C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\assets\data\newskill2.csv") 
        self.SKILLS_DB = list(self.data.columns.values)
        self.nlp = spacy.load('en_core_web_sm')
        self.matcher = Matcher(self.nlp.vocab)
    
    def __clean_text(self,resume_text):
        resume_text = re.sub('http\S+\s*', ' ', resume_text)  # remove URLs
        resume_text = re.sub('RT|cc', ' ', resume_text)  # remove RT and cc
        resume_text = re.sub('#\S+', '', resume_text)  # remove hashtags
        resume_text = re.sub('@\S+', '  ', resume_text)  # remove mentions
        resume_text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resume_text)  # remove punctuations
        resume_text = re.sub(r'[^\x00-\x7f]',r' ', resume_text) 
        resume_text = re.sub('\s+', ' ', resume_text)  # remove extra whitespace
        resume_text = resume_text.lower()  # convert to lowercase
        resume_text_tokens = word_tokenize(resume_text)  # tokenize
        filtered_text = [w for w in resume_text_tokens if not w in self.STOPWORDS]  # remove stopwords
        return ' '.join(filtered_text)
       
    def __extract_name(self,resume_text):
        #gave little inaccurate results
        nlp_text = self.nlp(resume_text)
        pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
        
        self.matcher.add('NAME',[pattern])
        
        matches = self.matcher(nlp_text)
        for match_id, start, end in matches:
            span = nlp_text[start:end+1]
            return span.text
        
        #the below code extracts all the nouns
        # Sentences = nltk.sent_tokenize(resume_text)
        # Tokens = []
        # for Sent in Sentences:
        #     Tokens.append(nltk.word_tokenize(Sent)) 
        # Words_List = [nltk.pos_tag(Token) for Token in Tokens]

        # Nouns_List = []

        # for List in Words_List:
        #     for Word in List:
        #         if re.match('[NN.*]', Word[1]):
        #             Nouns_List.append(Word[0])

        # Names = []
        # for Nouns in Nouns_List:
        #     if not wordnet.synsets(Nouns):
        #         Names.append(Nouns)

        # return Names
       
    
    def __extract_mobile_number(self,text):
        # phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
        # if phone:
        #     number = ''.join(phone[0])
        #     if len(number) > 10:
        #         return '+' + number
        #     else:
        #         return number
        numbers = []
        for match in phonenumbers.PhoneNumberMatcher(text, "IN"):  # Replace "IN" with the appropriate country code
            numbers.append(phonenumbers.format_number(match.number, phonenumbers.PhoneNumberFormat.E164))
        return numbers
        
    def __extract_email(self,email):
        email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
        if email:
            try:
                return email[0].split()[0].strip(';')
            except IndexError:
                return None
    
    def __extract_education(self,resume_text):
        nlp_text = self.nlp(resume_text)
        
        # Sentence Tokenizer
        nlp_text = [str(sent).strip() for sent in nlp_text.sents]
        edu = {}
        # Extract education degree
        for index, text in enumerate(nlp_text):
            for tex in text.split():
                # Replace all special symbols
                tex = re.sub(r'[?|$|.|!|,|(|)]', r'', tex)
                if tex.upper() in self.EDUCATION and tex not in self.STOPWORDS:
                    edu[tex] = text + nlp_text[index + 1]
                
        # Extract year
        education = []
        for key in edu.keys():
            year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
            if year:
                education.append((key, ''.join(year[0])))
            else:
                education.append(key)
        return education
    
    def __extract_skills(self,input_text):
        stop_words = set(nltk.corpus.stopwords.words('english'))
        word_tokens = nltk.tokenize.word_tokenize(input_text)

        # remove the stop words
        filtered_tokens = [w for w in word_tokens if w not in stop_words]

        # remove the punctuation
        filtered_tokens = [w for w in word_tokens if w.isalpha()]

        # generate bigrams and trigrams (such as artificial intelligence)
        bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

        # we create a set to keep the results in.
        found_skills = set()

        # we search for each token in our skills database
        for token in filtered_tokens:
            if token.lower() in self.SKILLS_DB:
                found_skills.add(token)

        # we search for each bigram and trigram in our skills database
        for ngram in bigrams_trigrams:
            if ngram.lower() in self.SKILLS_DB:
                found_skills.add(ngram)

        return found_skills
    
    def extractorData(self,file,ext): #
        text=""
        if ext=="docx": 
            temp = docx2txt.process(file)
            text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
            text = ' '.join(text)
        if ext=="pdf":
            for page in fitz.open(file):
                text = text + str(page.get_text())
            text = " ".join(text.split('\n'))
        #text = self.__clean_text(text)
        text1=text
        name = self.__extract_name(text)
        mobile_no = self.__extract_mobile_number(text)
        email = self.__extract_email(text)
        skills = self.__extract_skills(text)
        education1 = self.__extract_education(text)
        return (name,mobile_no,email,skills,education1,text1)

resumeExtractor = resumeExtraction()

print(resumeExtractor.extractorData(fitz.open(r"C:\Users\Asus\Desktop\Devansh_CV_latest .pdf"),"pdf"))

('Devansh Menaria Third', ['+919024196343'], 'menaria.1@iitj.ac.in', {'MySQL', 'process', 'communication', 'interactive', 'CSS', 'Software Engineering', 'Design', 'Architecture', 'JS', 'Statistics', 'Video', 'Java', 'Js', 'Illustrator', 'Python', 'HTML', 'Operating Systems', 'Programming', 'JavaScript', 'Algorithms', 'Pattern', 'PROGRAMMING', 'Editing', 'RESEARCH PROJECTS', 'Computer Science', 'DESIGN', 'Machine Learning', 'Flask', 'Database', 'Photoshop', 'scheduling', 'Technical', 'algorithms', 'Engineering', 'RESEARCH', 'analysis'}, [('BTech', '2024'), ('CBSE', '2018')], 'Last Updated on 1st December 2022 Devansh Menaria Third Year Undergraduate | Computer Science Engineering | IIT Jodhpur menaria.1@iitj.ac.in | 90241- 96343 EDUCATION IIT JODHPUR Third Year B.Tech in Computer Science Expected 2024 | Jodhpur, India Cum. GPA: 7.88 (Sem 4) DPS, UDAIPUR Class 12 | CBSE 2020 | Udaipur, India Percentage: 94.2% DPS, UDAIPUR Class 10 | CBSE 2018 | Udaipur, India Percentage: 95.2% COURSEWORK

In [3]:
fetched_data=resumeExtractor.extractorData(fitz.open(r"C:\Users\Asus\Desktop\Devansh_CV_latest .pdf"),"pdf")
fetched_data

('Devansh Menaria Third',
 ['+919024196343'],
 'menaria.1@iitj.ac.in',
 {'Algorithms',
  'Architecture',
  'CSS',
  'Computer Science',
  'DESIGN',
  'Database',
  'Design',
  'Editing',
  'Engineering',
  'Flask',
  'HTML',
  'Illustrator',
  'JS',
  'Java',
  'JavaScript',
  'Js',
  'Machine Learning',
  'MySQL',
  'Operating Systems',
  'PROGRAMMING',
  'Pattern',
  'Photoshop',
  'Programming',
  'Python',
  'RESEARCH',
  'RESEARCH PROJECTS',
  'Software Engineering',
  'Statistics',
  'Technical',
  'Video',
  'algorithms',
  'analysis',
  'communication',
  'interactive',
  'process',
  'scheduling'},
 [('BTech', '2024'), ('CBSE', '2018')],
 'Last Updated on 1st December 2022 Devansh Menaria Third Year Undergraduate | Computer Science Engineering | IIT Jodhpur menaria.1@iitj.ac.in | 90241- 96343 EDUCATION IIT JODHPUR Third Year B.Tech in Computer Science Expected 2024 | Jodhpur, India Cum. GPA: 7.88 (Sem 4) DPS, UDAIPUR Class 12 | CBSE 2020 | Udaipur, India Percentage: 94.2% DPS,

In [4]:
class resumeScreener:
    def __init__(self):
        self.setofStopWords = set(stopwords.words('english')+['``',"''"])
        self.max_length = 500
        self.trunc_type = 'post'
        self.padding_type = 'post'
    
    def __cleanResume(self,resumeText):
        resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
        resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
        resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
        resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
        resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
        resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
        resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
        resumeText = resumeText.lower()  # convert to lowercase
        resumeTextTokens = word_tokenize(resumeText)  # tokenize
        filteredText = [w for w in resumeTextTokens if not w in self.setofStopWords]  # remove stopwords
        return ' '.join(filteredText)
    
    def screenResume(self,text):
        # Get feature text tokenizer used for model training
        with open(r'C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\assets\tokenizer\feature_tokenizer.pickle', 'rb') as handle:
            feature_tokenizer = pickle.load(handle)

        # Get label encoding dictionary from model training
        with open(r'C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\assets\dictionary\dictionary.pickle', 'rb') as handle:
            encoding_to_label = pickle.load(handle)
    
        # Handle unknown label case and load original labels
        encoding_to_label[0] = 'unknown'
        with open(r"C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\assets\data\labels.json", "r") as read_file:
            original_labels = json.load(read_file)
        
        cleaned_input = self.__cleanResume(text)
        # Convert user input to padded sequence
        predict_sequences = feature_tokenizer.texts_to_sequences([cleaned_input])
        predict_padded = pad_sequences(predict_sequences, maxlen=self.max_length, padding=self.padding_type, truncating=self.trunc_type)
        predict_padded = np.array(predict_padded)
        
        # Load model and make prediction
        model = keras.models.load_model(r'C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\assets\model')
        prediction = model.predict(predict_padded)
        
        # Get encodings of top 5 results
        encodings = np.argpartition(prediction[0], -5)[-6:]
        encodings = encodings[np.argsort(prediction[0][encodings])]
        encodings = reversed(encodings)
        
        data = {}
        # Send results of top 5 encodings and confidences to output
        for encoding in encodings:
            label = encoding_to_label[encoding]
            probability = prediction[0][encoding] * 100
            probability = round(probability, 2)
            data[original_labels[label]]=probability
        
        if '.NET Developer' in data.keys():
            del(data['.NET Developer'])
        return data

resumeScreen = resumeScreener()



In [5]:
skillsPercentage = resumeScreen.screenResume(fetched_data[5])

1/1 [==============================] - 0s 234ms/step


In [6]:
skillsPercentage

{'Data Scientist': 23.2,
 'Web Designer': 7.9,
 'Civil Engineer': 6.58,
 'DevOps Engineer': 5.65,
 'Python Developer': 5.38}

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from sentence_transformers import SentenceTransformer

class jd_profile_comparison:
    def __init__(self):
        pass

    def __matcher(self,job_desc,resume_text):
        text=[resume_text,job_desc]
        cv=CountVectorizer()
        count_matrix=cv.fit_transform(text)
        matchper=cosine_similarity(count_matrix)[0][1] * 100
        return round(matchper,2)
        # model = SentenceTransformer('all-mpnet-base-v2')
        # #Encoding:
        # score = 0
        # sen = job_desc+resume_text
        # sen_embeddings = model.encode(sen)
        # for i in range(len(job_desc)):
        #     if job_desc[i] in resume_text:
        #         score += 1
        #     else:
        #         if max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job_desc):])[0]) >= 0.4:
        #             score += max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job_desc):])[0])
        # score = score/len(job_desc)  
        # return round(score,3)
    
    def match(self,jd,resumetext):
        return self.__matcher(jd,resumetext)

obj_jd_profile_comparison = jd_profile_comparison()

c:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def extractData(file,ext):
    text=""
    if ext=="docx": 
        temp = docx2txt.process(file)
        text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
        text = ' '.join(text)
    if ext=="pdf":
        for page in fitz.open(file):
            text = text + str(page.get_text())
        text = " ".join(text.split('\n'))
    return text

In [9]:
fetched_job=extractData("C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\Implementation\job_desc.docx","docx")
fetched_job

'Job details: Develop information systems by architecting, designing, and implementing advanced software solutions. Create software by studying business needs, conferring with process owners, and examining systems flow, data usage, and work processes. Author documentation, flowcharts, layouts, diagrams, charts, code comments, and clear code for solutions development. Bachelor’s and/or master’s degree in computer science, computer engineering, data sciences, or related technical discipline 5+ years of professional software development experience Experience with computer vision or nlp Proficiency in Java or C++'

In [10]:
result1=obj_jd_profile_comparison.match(fetched_job,fetched_data[5])

In [11]:
fetched_data[5]

'Last Updated on 1st December 2022 Devansh Menaria Third Year Undergraduate | Computer Science Engineering | IIT Jodhpur menaria.1@iitj.ac.in | 90241- 96343 EDUCATION IIT JODHPUR Third Year B.Tech in Computer Science Expected 2024 | Jodhpur, India Cum. GPA: 7.88 (Sem 4) DPS, UDAIPUR Class 12 | CBSE 2020 | Udaipur, India Percentage: 94.2% DPS, UDAIPUR Class 10 | CBSE 2018 | Udaipur, India Percentage: 95.2% COURSEWORK UNDERGRADUATE Pattern Recognition and Machine Learning Data Structures and Algorithms Operating Systems Software Engineering Database Management Systems Computer Architecture Design analysis of Algorithms Principles of Programming Languages Maths For Computing Probability and Statistics Mathematics- I, II SKILLS PROGRAMMING Proficient: •Python • C++ Familiar: •Java • Java Script WEB •HTML/CSS/Bootstrap •Django •Express Js •MongoDB •Node.js •React OTHERS •Adobe Premiere Pro •Adobe After Effects •Adobe Photoshop •Adobe Illustrator •Adobe Audition RESEARCH & PROJECTS CPU SCHED

In [12]:
result1

29.14

In [13]:
data = pd.read_csv(r"C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\assets\data\UpdatedResumeDataSet.csv", engine='python')
data

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
...,...,...
957,Testing,Computer Skills: â¢ Proficient in MS office (...
958,Testing,â Willingness to accept the challenges. â ...
959,Testing,"PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne..."
960,Testing,COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...


In [14]:
data = data[data.Category != 'Testing']
data_size = len(data)

In [15]:
stopwords_set = set(stopwords.words('english')+['``',"''"])
def clean_text(resume_text):
    resume_text = re.sub('http\S+\s*', ' ', resume_text)  # remove URLs
    resume_text = re.sub('RT|cc', ' ', resume_text)  # remove RT and cc
    resume_text = re.sub('#\S+', '', resume_text)  # remove hashtags
    resume_text = re.sub('@\S+', '  ', resume_text)  # remove mentions
    resume_text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resume_text)  # remove punctuations
    resume_text = re.sub(r'[^\x00-\x7f]',r' ', resume_text) 
    resume_text = re.sub('\s+', ' ', resume_text)  # remove extra whitespace
    resume_text = resume_text.lower()  # convert to lowercase
    resume_text_tokens = word_tokenize(resume_text)  # tokenize
    filtered_text = [w for w in resume_text_tokens if not w in stopwords_set]  # remove stopwords
    return ' '.join(filtered_text)

In [16]:
# Print a sample original resume
print('--- Original resume ---')
print(data['Resume'][0])

--- Original resume ---
Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details 

Data Science Assurance Associate 

Data Science Assurance Associate - Ernst & Young LLP
Skill Details 
JAVASCRIPT- Exprience - 24 months
jQuery- Exprience - 24 months
Python- Exprience - 24 monthsCompany Details 
company - Ernst & Young LLP
description - Fraud Investigations and 

In [17]:
# Print the same resume after text cleaning
data['cleaned_resume'] = data.Resume.apply(lambda x: clean_text(x))

print('--- Cleaned resume ---')
print(data['cleaned_resume'][0])

--- Cleaned resume ---
skills programming languages python pandas numpy scipy scikit learn matplotlib sql java javascript jquery machine learning regression svm na bayes knn random forest decision trees boosting techniques cluster analysis word embedding sentiment analysis natural language processing dimensionality reduction topic modelling lda nmf pca neural nets database visualizations mysql sqlserver cassandra hbase elasticsearch d3 js dc js plotly kibana matplotlib ggplot tableau others regular expression html css angular 6 logstash kafka python flask git docker computer vision open cv understanding deep learning education details data science assurance associate data science assurance associate ernst young llp skill details javascript exprience 24 months jquery exprience 24 months python exprience 24 monthscompany details company ernst young llp description fraud investigations dispute services assurance technology assisted review tar technology assisted review assists elerating r

C:\Users\Asus\AppData\Local\Temp\ipykernel_1680\887221286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_resume'] = data.Resume.apply(lambda x: clean_text(x))


In [18]:
data

,Category,Resume,cleaned_resume
0,Data Science,Skills * Programming Languages: Python (pandas...,skills programming languages python pandas num...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education details may 2013 may 2017 b e uit rg...
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",areas interest deep learning control system de...
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skills r python sap hana tableau sap hana sql ...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education details mca ymcaust faridabad haryan...
...,...,...,...
887,Blockchain,Hobbies â¢ Playing Chess â¢ Solving Rubik's ...,hobbies playing chess solving rubik cube watch...
888,Blockchain,Skills Strong CS fundamentals and problem solv...,skills strong cs fundamentals problem solving ...
889,Blockchain,"KEY SKILLS: Programing languages: C, C++, Pyth...",key skills programing languages c c python ape...
890,Blockchain,"SOFTWARE SKILLS: Languages: C, C++ & java Oper...",software skills languages c c java operating s...


In [19]:
result_cv=obj_jd_profile_comparison.match(fetched_job,fetched_data[5])
result_cv

29.14

In [20]:
result_dict={}
for i in range(data_size):
    result=0
    result=obj_jd_profile_comparison.match(fetched_job,data['cleaned_resume'][i])
    result_dict[i]=result

result_dict

{0: 12.58,
 1: 3.87,
 2: 9.67,
 3: 14.15,
 4: 7.13,
 5: 7.34,
 6: 21.3,
 7: 14.09,
 8: 16.7,
 9: 16.93,
 10: 12.58,
 11: 3.87,
 12: 9.67,
 13: 14.15,
 14: 7.13,
 15: 7.34,
 16: 21.3,
 17: 14.09,
 18: 16.7,
 19: 16.93,
 20: 12.58,
 21: 3.87,
 22: 9.67,
 23: 14.15,
 24: 7.13,
 25: 7.34,
 26: 21.3,
 27: 14.09,
 28: 16.7,
 29: 16.93,
 30: 12.58,
 31: 3.87,
 32: 9.67,
 33: 14.15,
 34: 7.13,
 35: 7.34,
 36: 21.3,
 37: 14.09,
 38: 16.7,
 39: 16.93,
 40: 6.34,
 41: 6.91,
 42: 4.71,
 43: 4.55,
 44: 4.55,
 45: 7.38,
 46: 6.63,
 47: 6.82,
 48: 2.63,
 49: 6.46,
 50: 19.13,
 51: 6.34,
 52: 6.91,
 53: 4.71,
 54: 4.55,
 55: 4.55,
 56: 7.38,
 57: 6.63,
 58: 6.82,
 59: 2.63,
 60: 6.46,
 61: 19.13,
 62: 6.34,
 63: 6.91,
 64: 4.71,
 65: 4.55,
 66: 4.55,
 67: 7.38,
 68: 6.63,
 69: 6.82,
 70: 2.63,
 71: 6.46,
 72: 19.13,
 73: 6.34,
 74: 6.91,
 75: 4.71,
 76: 4.55,
 77: 4.55,
 78: 7.38,
 79: 6.63,
 80: 6.82,
 81: 2.63,
 82: 6.46,
 83: 19.13,
 84: 2.34,
 85: 2.95,
 86: 4.79,
 87: 3.22,
 88: 6.86,
 89: 5.05,


In [21]:
sorted_dict_results = dict(sorted(result_dict.items(), key=lambda x: x[1], reverse=True))

top_5_items = dict(list(sorted_dict_results.items())[:5])

print(type(top_5_items))

<class 'dict'>


In [22]:
top_5_items

{6: 21.3, 16: 21.3, 26: 21.3, 36: 21.3, 50: 19.13}

In [23]:
result_df=pd.DataFrame(top_5_items.items(), columns=['ResumeID', 'Matching Percentage'])
result_df

,ResumeID,Matching Percentage
0,6,21.30
1,16,21.30
2,26,21.30
3,36,21.30
4,50,19.13


## Using Sentence transformer(BERT) for encoding and calculating matching percentage using sklearn Cosine Similarity

In [24]:
def matcher(job_desc,resume_text):
        text=[job_desc,resume_text]
        model = SentenceTransformer('all-mpnet-base-v2')
        # sen = job_desc+fetched_data[5]
        sen_embeddings = model.encode(text)
        # (sen_embeddings).shape
        return(cosine_similarity([sen_embeddings[0],sen_embeddings[1]])[0][1])*100
        

In [25]:
text=[fetched_job,fetched_data[5]]
model = SentenceTransformer('all-mpnet-base-v2')
# sen = job_desc+fetched_data[5]
sen_embeddings = model.encode(text)
(sen_embeddings).shape
(cosine_similarity([sen_embeddings[0],sen_embeddings[1]])[0][1])*100

44.69018578529358

In [26]:
result_dict_2={}
for i in range(data_size):
    result=0
    result=matcher(fetched_job,data['cleaned_resume'][i])
    result_dict_2[i]=result

result_dict_2

{0: 45.70041596889496,
 1: 43.4290736913681,
 2: 55.628931522369385,
 3: 65.97411632537842,
 4: 56.498003005981445,
 5: 48.541638255119324,
 6: 64.64754343032837,
 7: 51.64064168930054,
 8: 47.55661487579346,
 9: 56.20037317276001,
 10: 45.70041596889496,
 11: 43.4290736913681,
 12: 55.628931522369385,
 13: 65.97411632537842,
 14: 56.498003005981445,
 15: 48.541638255119324,
 16: 64.64754343032837,
 17: 51.64064168930054,
 18: 47.55661487579346,
 19: 56.20037317276001,
 20: 45.70041596889496,
 21: 43.4290736913681,
 22: 55.628931522369385,
 23: 65.97411632537842,
 24: 56.498003005981445,
 25: 48.541638255119324,
 26: 64.64754343032837,
 27: 51.64064168930054,
 28: 47.55661487579346,
 29: 56.20037317276001,
 30: 45.70041596889496,
 31: 43.4290736913681,
 32: 55.628931522369385,
 33: 65.97411632537842,
 34: 56.498003005981445,
 35: 48.541638255119324,
 36: 64.64754343032837,
 37: 51.64064168930054,
 38: 47.55661487579346,
 39: 56.20037317276001,
 40: 42.76409447193146,
 41: 53.8203537464

In [27]:
sorted_dict_results_2 = dict(sorted(result_dict_2.items(), key=lambda x: x[1], reverse=True))

top_5_items_2 = dict(list(sorted_dict_results_2.items())[:5])

print(type(top_5_items_2))

<class 'dict'>


In [28]:
result_df_2=pd.DataFrame(top_5_items_2.items(), columns=['ResumeID', 'Matching Percentage'])
result_df_2

,ResumeID,Matching Percentage
0,404,68.463600
1,418,68.463600
2,786,67.095947
3,791,67.095947
4,796,67.095947


Using Sklearn's cosine similarity takes about 5.7 seconds per resume to screen, took a total of 48 mins to screen 891 resumes

## Word Mover's  with word2vec model to calculate percentage similarity

 Note: This function calculated WMD without tokenization

In [29]:
# import os
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
# from nltk.corpus import stopwords
# from nltk import download
# import gensim
# from gensim.models import Word2Vec
# from numba import jit
# download('stopwords')  # Download stopwords list.
# # @jit#(target="cuda")
# def matcher_2(job_desc, res_text):
#     # Remove stopwords.
#     stop_words = stopwords.words('english')
#     job_desc = [w for w in job_desc if w not in stop_words]
#     res_text = [w for w in res_text if w not in stop_words]
#     model = gensim.models.KeyedVectors.load_word2vec_format('C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\Implementation\GoogleNews-vectors-negative300.bin.gz', binary=True)
#     model.init_sims(replace=True)  # Normalizes the vectors in the word2vec class.
#     distance = model.wmdistance(job_desc, res_text)
#     return distance  



In [30]:
# matcher_2(fetched_job,fetched_data[5])

In [31]:
# result_dict_3={}
# for i in range(data_size):
#     result=0
#     result=matcher_2(fetched_job,data['cleaned_resume'][i])
#     result_dict_3[i]=result

# result_dict_3

WMD takes about 90 seconds to screen 1 resume

## Using Sentence transformer(BERT) for encoding and calculating matching percentage using Scipy Cosine Similarity

In [32]:
from scipy.spatial import distance
text=[fetched_job,fetched_data[5]]
model = SentenceTransformer('all-mpnet-base-v2')
# sen = job_desc+fetched_data[5]
sen_embeddings = model.encode(text)
(sen_embeddings).shape
(distance.cosine(sen_embeddings[0],sen_embeddings[1]))*100

55.30981123447418

In [33]:
from scipy.spatial import distance

def matcher_4(job_desc,resume_text):
        text=[job_desc,resume_text]
        model = SentenceTransformer('all-mpnet-base-v2')
        # sen = job_desc+fetched_data[5]
        sen_embeddings = model.encode(text)
        # (sen_embeddings).shape
        return(distance.cosine(sen_embeddings[0],sen_embeddings[1]))*100
        

In [34]:
result_dict_4={}
for i in range(data_size):
    result=0
    result=matcher_4(fetched_job,data['cleaned_resume'][i])
    result_dict_4[i]=result

result_dict_4

KeyboardInterrupt: 

In [ ]:
sorted_dict_results_4 = dict(sorted(result_dict_4.items(), key=lambda x: x[1], reverse=True))

top_5_items_4 = dict(list(sorted_dict_results_4.items())[:5])

print(type(top_5_items_4))

<class 'dict'>


In [ ]:
result_df_4=pd.DataFrame(top_5_items_4.items(), columns=['ResumeID', 'Matching Percentage'])
result_df_4


,ResumeID,Matching Percentage
0,266,90.90351
1,272,90.90351
2,278,90.90351
3,284,90.90351
4,290,90.90351


Scipy cosine similarity along with Bert transformer shows a very high matching percentage but takes overall 48 mins to screen 891 resumes

## Using Sentence transformer(BERT{all-MiniLM-L6-v2}) for encoding and calculating matching percentage using Sklearn Cosine Similarity

In [38]:
def matcher_5(job_desc,resume_text):
        text=[job_desc,resume_text]
        model = SentenceTransformer('all-MiniLM-L6-v2')
        # sen = job_desc+fetched_data[5]
        sen_embeddings = model.encode(text)
        # (sen_embeddings).shape
        return(cosine_similarity([sen_embeddings[0],sen_embeddings[1]])[0][1])*100
        

In [40]:
text=[fetched_job,fetched_data[5]]
model = SentenceTransformer('all-MiniLM-L6-v2')
# sen = job_desc+fetched_data[5]
sen_embeddings = model.encode(text)
(sen_embeddings).shape
(cosine_similarity([sen_embeddings[0],sen_embeddings[1]])[0][1])*100

38.332659006118774

In [41]:
result_dict_5={}
for i in range(data_size):
    result=0
    result=matcher_5(fetched_job,data['cleaned_resume'][i])
    result_dict_5[i]=result

result_dict_5

{0: 40.07311463356018,
 1: 32.90212154388428,
 2: 40.32559394836426,
 3: 39.92169201374054,
 4: 48.866093158721924,
 5: 45.89669704437256,
 6: 41.048941016197205,
 7: 38.672298192977905,
 8: 38.76413106918335,
 9: 40.71744084358215,
 10: 40.07311463356018,
 11: 32.90212154388428,
 12: 40.32559394836426,
 13: 39.92169201374054,
 14: 48.866093158721924,
 15: 45.89669704437256,
 16: 41.048941016197205,
 17: 38.672298192977905,
 18: 38.76413106918335,
 19: 40.71744084358215,
 20: 40.07311463356018,
 21: 32.90212154388428,
 22: 40.32559394836426,
 23: 39.92169201374054,
 24: 48.866093158721924,
 25: 45.89669704437256,
 26: 41.048941016197205,
 27: 38.672298192977905,
 28: 38.76413106918335,
 29: 40.71744084358215,
 30: 40.07311463356018,
 31: 32.90212154388428,
 32: 40.32559394836426,
 33: 39.92169201374054,
 34: 48.866093158721924,
 35: 45.89669704437256,
 36: 41.048941016197205,
 37: 38.672298192977905,
 38: 38.76413106918335,
 39: 40.71744084358215,
 40: 48.94891381263733,
 41: 31.170955

In [42]:
sorted_dict_results_5 = dict(sorted(result_dict_5.items(), key=lambda x: x[1], reverse=True))

top_5_items_5 = dict(list(sorted_dict_results_5.items())[:5])

print(type(top_5_items_5))

<class 'dict'>


In [43]:
result_df_5=pd.DataFrame(top_5_items_5.items(), columns=['ResumeID', 'Matching Percentage'])
result_df_5

,ResumeID,Matching Percentage
0,435,59.361297
1,441,59.361297
2,447,59.361297
3,453,59.361297
4,50,56.417882


## Using glove with cosine similarity

In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

PATH_TO_GLOVE = "C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\Implementation\glove.840B.300d\glove.840B.300d.txt"

tmp_file = "C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\Implementation\GoogleNews-vectors-negative300.bin.gz"
glove2word2vec(PATH_TO_GLOVE, tmp_file)
# glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)




C:\Users\Asus\AppData\Local\Temp\ipykernel_30880\2120685729.py:8: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(PATH_TO_GLOVE, tmp_file)
2023-06-21 19:16:51,146 : WARNING : duplicate word '����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������' in word2vec file, ignoring all but first


KeyboardInterrupt: 

## Checking the predictions of models manually

In [35]:
print(fetched_job)

Job details: Develop information systems by architecting, designing, and implementing advanced software solutions. Create software by studying business needs, conferring with process owners, and examining systems flow, data usage, and work processes. Author documentation, flowcharts, layouts, diagrams, charts, code comments, and clear code for solutions development. Bachelor’s and/or master’s degree in computer science, computer engineering, data sciences, or related technical discipline 5+ years of professional software development experience Experience with computer vision or nlp Proficiency in Java or C++


Model 1

In [36]:
print(data['Category'][6])
print(data['Category'][16])
print(data['Category'][26])
print(data['Category'][36])
print(data['Category'][50])

Data Science
Data Science
Data Science
Data Science
HR


Model 2

In [37]:
print(data['Category'][404])
print(data['Category'][418])
print(data['Category'][786])
print(data['Category'][791])
print(data['Category'][796])


Business Analyst
Business Analyst
ETL Developer
ETL Developer
ETL Developer


Model 4

In [ ]:
print(data['Category'][266])
print(data['Category'][272])
print(data['Category'][278])
print(data['Category'][284])
print(data['Category'][290])

Health and fitness
Health and fitness
Health and fitness
Health and fitness
Health and fitness


Model 5

In [44]:
print(data['Category'][435])
print(data['Category'][441])
print(data['Category'][447])
print(data['Category'][453])
print(data['Category'][50])

SAP Developer
SAP Developer
SAP Developer
SAP Developer
HR


Model 6

In [66]:
print(data['Category'][50])
print(data['Category'][61])
print(data['Category'][72])
print(data['Category'][83])
print(data['Category'][435])

HR
HR
HR
HR
SAP Developer


# Approach II (Targetting just skills extracted instead of supplying complete resume and job descriptions as text)

In [52]:
fetch_job=resumeExtractor.extractorData((r"C:\Workspace\Projects\Resume Summarizer\ResumeRanker_Shared\env\ResumeRanker_Shared\Implementation\job_desc.docx"),"docx")
fetch_job

(None,
 [],
 None,
 {'Java',
  'computer science',
  'documentation',
  'engineering',
  'process',
  'technical'},
 [],
 'Job details: Develop information systems by architecting, designing, and implementing advanced software solutions. Create software by studying business needs, conferring with process owners, and examining systems flow, data usage, and work processes. Author documentation, flowcharts, layouts, diagrams, charts, code comments, and clear code for solutions development. Bachelor’s and/or master’s degree in computer science, computer engineering, data sciences, or related technical discipline 5+ years of professional software development experience Experience with computer vision or nlp Proficiency in Java or C++')

In [62]:
fetch_job_skills=list(fetch_job[3])
type(fetch_job_skills)
fetch_data_skills=list(fetched_data[3])
type(fetch_data_skills)

list

In [63]:
text=[fetch_job_skills,fetch_data_skills]
model = SentenceTransformer('all-MiniLM-L6-v2')
# sen = job_desc+fetched_data[5]
sen_embeddings = model.encode(text)
(sen_embeddings).shape
(cosine_similarity([sen_embeddings[0],sen_embeddings[1]])[0][1])*100

51.15554332733154

In [64]:
result_dict_6={}
for i in range(data_size):
    result=0
    result=matcher_5(fetch_job_skills,data['cleaned_resume'][i])
    result_dict_6[i]=result

result_dict_6

{0: 43.05124282836914,
 1: 32.539376616477966,
 2: 41.83146357536316,
 3: 35.34024953842163,
 4: 47.32142388820648,
 5: 45.67066431045532,
 6: 40.717458724975586,
 7: 37.57208585739136,
 8: 40.71521759033203,
 9: 34.33138728141785,
 10: 43.05124282836914,
 11: 32.539376616477966,
 12: 41.83146357536316,
 13: 35.34024953842163,
 14: 47.32142388820648,
 15: 45.67066431045532,
 16: 40.717458724975586,
 17: 37.57208585739136,
 18: 40.71521759033203,
 19: 34.33138728141785,
 20: 43.05124282836914,
 21: 32.539376616477966,
 22: 41.83146357536316,
 23: 35.34024953842163,
 24: 47.32142388820648,
 25: 45.67066431045532,
 26: 40.717458724975586,
 27: 37.57208585739136,
 28: 40.71521759033203,
 29: 34.33138728141785,
 30: 43.05124282836914,
 31: 32.539376616477966,
 32: 41.83146357536316,
 33: 35.34024953842163,
 34: 47.32142388820648,
 35: 45.67066431045532,
 36: 40.717458724975586,
 37: 37.57208585739136,
 38: 40.71521759033203,
 39: 34.33138728141785,
 40: 46.720218658447266,
 41: 29.898402094

In [65]:
sorted_dict_results_6 = dict(sorted(result_dict_6.items(), key=lambda x: x[1], reverse=True))

top_5_items_6 = dict(list(sorted_dict_results_6.items())[:5])

print(type(top_5_items_6))
result_df_6=pd.DataFrame(top_5_items_6.items(), columns=['ResumeID', 'Matching Percentage'])
result_df_6

<class 'dict'>


,ResumeID,Matching Percentage
0,50,56.201226
1,61,56.201226
2,72,56.201226
3,83,56.201226
4,435,54.212582
